In [6]:
infile = "/data/Data/yntn/data_cleaned.log"
outfile = "/data/Data/yntn/data_out.csv"
emoji_path = '/data/Data/feed_data/new_cat'

In [7]:
with open(infile, "r") as f:
    lines = f.readlines()   

In [8]:
import json

# read file
with open('emoji2code.json', 'r') as myfile:
    data = myfile.read()

# parse file
obj = json.loads(data)
emoji = obj.keys()

In [9]:
from pathlib import Path

emoji_dict = {}

for file_dir in Path(emoji_path).glob('**/*.png'):
    if (str(file_dir.parent.stem) in emoji_dict):  
        emoji_dict[str(file_dir.parent.stem)].append(str(file_dir.stem))
    else:
        emoji_dict[str(file_dir.parent.stem)] = [str(file_dir.stem)]

In [14]:
import operator
import csv
import re, string
 
with open(outfile, "w") as f:
    writer = csv.writer(f)
    myData = [['text','label']]
    for line in lines:
        if ("feed at" in line):
            continue        
                
        count = {}
        st = line
        for phrase in emoji:
            if (phrase in st):
                count[obj[phrase]] = 0
                while (phrase in st):
                    count[obj[phrase]] = count[obj[phrase]] + 1
                    st = st.replace(phrase,'',1)                    
        emoji_max = max(count.items(), key=operator.itemgetter(1))[0] #string
        
        st = st.lower()
        words = st.split()
        words = re.split(r'\W+', st)
        text = ''        
        text = ' '.join(word for word in words)
        text = text.strip()
        
        if ('ttt' in text):
            continue
                
        if (text == "") or (type(text) == float) or (str(text) =='nan'): 
            continue         
        for etype in emoji_dict.keys():
            if emoji_max in emoji_dict[etype]:
                myData.append([text, etype.lower()])
                break
                
    writer.writerows(myData)

In [ ]:
import pandas as pd 
data = pd.read_csv('/data/Data/yntn/data_out.csv') 
data